# Libraries

In [207]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import datetime
import glob
import os

# Connections

In [208]:
from carvana_sql import SQLServer
db_conn = SQLServer.connect('PEOPLEOPSPROD,1439')

In [209]:
cnx_str = "Driver={ODBC Driver 17 for SQL Server};Server=PEOPLEOPSPROD,1439;Database=PEOPLEOPS;Trusted_Connection=yes;"
import pyodbc
odbcconn = pyodbc.connect(cnx_str)
odbcquery = 'Select Top 10 * from PeopleOps.tenstreet.FactDriver'
odbcdf = pd.read_sql_query(odbcquery, odbcconn)

C:\Users\E102068\AppData\Local\Temp\ipykernel_16284\4105226840.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  odbcdf = pd.read_sql_query(odbcquery, odbcconn)


In [210]:
# Todays Date - Used in multiple SQL queries
# Change the value of today if you'd like to run the code at a different time

# date_today=pd.to_datetime('2024-03-01')

date_today=pd.to_datetime(datetime.date.today())

In [211]:
from carvana_sql import Snowflake

SnowflakeWarehouse = 'PEOPLEOPS_ANALYTICS'
SnowflakeRole = 'PEOPLEOPS_ANALYTICS'
SnowflakeEmail = 'anish.middela@carvana.com'
SnowflakeDatabase = "PEOPLEOPS_SANDBOX"
Snowflakeschema="SITERISK"

db_snowflake = Snowflake.connect(
                username=SnowflakeEmail,                
                role=SnowflakeRole,
                warehouse=SnowflakeWarehouse)

db_snowflake_write = Snowflake.connect(
                username=SnowflakeEmail,                
                role=SnowflakeRole,
                warehouse=SnowflakeWarehouse,
                database=SnowflakeDatabase,
                schema= Snowflakeschema )

# Glint

### Location Files

In [212]:
# Local CSV Pull

# # Files to pull location demographic csv
# location_path="Glint Reports"

# # Level 1 csv files
# location_files= glob.glob(os.path.join(location_path, "*\*.csv"))

# # Loop over lvl 2 subdirectory csv files
# #location_files= glob.glob(os.path.join(location_path, "**\*.csv"), recursive= True)

In [213]:
# Netwrok csv pull
network_location_path="X:\People_Operations\Secured\Talent Analytics\Anish Middela\SiteRisk"

# Level 1 csv files
network_location_files= glob.glob(os.path.join(network_location_path, "**\*.csv"),recursive=True)

In [214]:
network_location_files

["X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Glint Reports\\Q1'24\\ADESA\\glint_Q1'24_ADESA.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Glint Reports\\Q1'24\\Carvana\\glint_Carvana_Q1'24_Logistics.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Glint Reports\\Q1'24\\Carvana\\glint_Carvana_Q1'24_Market Operations.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Glint Reports\\Q1'24\\Carvana\\glint_Carvana_Q1'24_Post Production.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Glint Reports\\Q1'24\\Carvana\\glint_Carvana_Q1'24_Reconditioning.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Glint Reports\\Q1'24\\Carvana\\glint_Carvana_Q1'24_Regulatory Operations.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Glint Reports\\Q1'24\\Carvana\\glint_Carva

In [215]:
glint_files = []
injury_files = []
overall_files=[]


for path in network_location_files:
    if "Overall" in path:
        overall_files.append(path)
    elif "Glint" in path:
        glint_files.append(path)
    elif "Injuries" in path:
        injury_files.append(path)

In [216]:
# files=os.listdir("C:/Users/E102068/Desktop/SiteRisk Dashboard/Glint Reports/Q1_24")

# # Files with overall data
# main_path="Glint Reports\Q1_24\Overall"

# main_files= glob.glob(main_path+"\*.csv")

In [217]:
glint_files

["X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Glint Reports\\Q1'24\\ADESA\\glint_Q1'24_ADESA.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Glint Reports\\Q1'24\\Carvana\\glint_Carvana_Q1'24_Logistics.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Glint Reports\\Q1'24\\Carvana\\glint_Carvana_Q1'24_Market Operations.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Glint Reports\\Q1'24\\Carvana\\glint_Carvana_Q1'24_Post Production.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Glint Reports\\Q1'24\\Carvana\\glint_Carvana_Q1'24_Reconditioning.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Glint Reports\\Q1'24\\Carvana\\glint_Carvana_Q1'24_Regulatory Operations.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Glint Reports\\Q1'24\\Carvana\\glint_Carva

In [218]:
location_df = pd.DataFrame()
# Appending all the csv files into one df
for file in glint_files:
    df_temp=pd.read_csv(file)
    df_temp['JobFamilyGroup']= file.split('_')[-1].split('.')[0]
    df_temp['Quarter']=file.split('_')[-2]

    location_df=pd.concat([location_df,df_temp],ignore_index=True)

    #df_temp['Quarter']=file.split('\\')[1] #Used this for Local Located files rather than network
        # filename=file.split('_')[-1].split('.')[0]

In [219]:
# Select only neccesary columns
location_df=location_df.iloc[:,[0,5,6,15,16]]

# Rename
location_df= location_df.rename(columns={location_df.columns[1]:'Score',location_df.columns[2]:'eSat Change'})

# Filter out Location ==ALL
location_df = location_df[~(location_df['Location'] == 'All')]

In [220]:
# Convert 'Score' column to numeric, errors='coerce' will replace non-numeric values with NaN
location_df['Score'] = pd.to_numeric(location_df['Score'], errors='coerce')
location_df['eSat Change'] = pd.to_numeric(location_df['eSat Change'], errors='coerce')

# Fill na ==0
location_df=location_df.fillna(0)

location_df.head(5)

,Location,Score,eSat Change,JobFamilyGroup,Quarter
0,AL Birmingham - Sollie Recon Center,74.0,-7.0,ADESA,Q1'24
1,AL Remote,0.0,0.0,ADESA,Q1'24
3,AR Little Rock - McNeill,75.0,-1.0,ADESA,Q1'24
4,AZ Mesa,0.0,0.0,ADESA,Q1'24
5,AZ Phoenix - Beck IC,83.0,-4.0,ADESA,Q1'24


In [221]:
location_df.groupby('Quarter').size()

Quarter
Q1'24    400
Q1'25    407
Q2'24    417
Q3'24    415
Q4'24    398
dtype: int64

### Necessary JobFamilyGroups

In [222]:
necessary_JobFamilyGroups= location_df['JobFamilyGroup'].unique().tolist()

print(necessary_JobFamilyGroups)

['ADESA', 'Logistics', 'Market Operations', 'Post Production', 'Reconditioning', 'Regulatory Operations', 'Wholesale Operations']


# Injuries

In [223]:
# Old Logic
# injury_path=("Injuries")

# injury_files= glob.glob(os.path.join(injury_path, "*.csv"))

# injuries_df= glob.glob(injury_path+'\*.csv')
# injuries_df=pd.read_csv("C:/Users/E102068/Desktop/SiteRisk Dashboard/Injuries/injuries_report_Q1'24.csv")

In [224]:
injury_files

["X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Injuries\\injuries_report_Q1'24.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Injuries\\injuries_report_Q1'25.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Injuries\\injuries_report_Q2'24.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Injuries\\injuries_report_Q3'24.csv",
 "X:\\People_Operations\\Secured\\Talent Analytics\\Anish Middela\\SiteRisk\\Injuries\\injuries_report_Q4'24.csv"]

In [225]:
injury_df_main= pd.DataFrame()
# Appending all the csv files into one df
for file in injury_files:
    df_temp_injury=pd.read_csv(file)
    df_temp_injury['Quarter']=file.split('_')[-1].split('.')[0]
        # filename=file.split('_')[-1].split('.')[0]
    injury_df_main=pd.concat([injury_df_main,df_temp_injury],ignore_index=True)

In [226]:
# Pull JobProfile Detail from workday - TABLE EMPLOYEE HISTORY
# JobFamily_df = pd.read_sql_query("""
                             
# SELECT distinct JobProfile,JobFamilyGroup
# FROM [PeopleOps].[workday].[tblEmployeeHistory]
# WHERE Company NOT like '%ADESA%'
# AND JobFamilyGroup NOT like '%ADESA%'

# """,odbcconn)

In [227]:
# Merging with Workday Jobfamily to get more accurate information
# injury_df_main= pd.merge(injury_df_main,JobFamily_df, how='left', on='JobProfile')

# injury_df_main=injury_df_main.copy()

# injury_df_check= pd.merge(injury_df,JobFamily_df, how='left', left_on='JobProfile',right_on='JobProfileName')

In [229]:
# Renaming the column
# injury_df_main=injury_df_main.rename(columns={'JobFamilyGroup_y':'JobFamilyGroup'})
injury_df_main.columns = injury_df_main.columns.str.strip()
# Wherever their are NA then pointing it to Company
injury_df_main['JobFamilyGroup'] = np.where(injury_df_main['JobFamilyGroup'].isna(),injury_df_main['COMPANY'],injury_df_main['JobFamilyGroup'])


# Renaming Col
injury_df_snowflake= injury_df_main.rename(columns={'WORK_LOCATION':'Location','BODYPART2':'Injuries'})

In [230]:
injury_df_snowflake.head(5)

,Injuries,ACLAIMANT_LOCATION_NAME,Department/Group,ACTIONCODE2,STATE_OF_JURISDICTION,Location,COMPANY,JobFamily,JobFamilyGroup,JobProfile,...,New_Vehicle_Type,OCCURRENCE_NUMBER,Open Claim Flag,PROPERTY_ID_C,TITLE_C,VEHICLE_ID_NUMBER_C,OCCURRED_DAY,EMPLOYEE_ID,NATURE,CAUSE_CODE
0,Back,Los Angeles (#1),Null in JobFamilyGroup,General - Walking on Company Property (No Othe...,CA,ADESA Riverside,ADESA,NaN,ADESA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Back,Salt Lake City,Null in JobFamilyGroup,Personal Illnesss/Injury,UT,Salt Lake,ADESA,NaN,ADESA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"['Shoulder(s)', 'Ankle']",Los Angeles Adesa (Nino Way),Null in JobFamilyGroup,General - Walking on Company Property (No Othe...,CA,Los Angeles,ADESA,NaN,ADESA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Knee,Cinc-Dayton,Null in JobFamilyGroup,General - Walking on Company Property (No Othe...,OH,ADESA Cincinnati,ADESA,NaN,ADESA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Finger(s),Kansas City,Null in JobFamilyGroup,Vehicle Detail Exterior/Interior,MO,"Belton, MO",ADESA,NaN,ADESA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [231]:
injury_df_snowflake.groupby('Quarter').size()

Quarter
Q1'24    346
Q1'25    533
Q2'24    398
Q3'24    447
Q4'24    419
dtype: int64

### Snowflake - Injury

In [232]:
injury_df_snowflake = injury_df_snowflake[['Injuries','Location','JobFamilyGroup','JobProfile','Quarter','COMPANY','STATUS','CASE_NUMBER']]

injury_df_snowflake.head(5)

,Injuries,Location,JobFamilyGroup,JobProfile,Quarter,COMPANY,STATUS,CASE_NUMBER
0,Back,ADESA Riverside,ADESA,NaN,Q1'24,ADESA,submitted,ADESA-7740
1,Back,Salt Lake,ADESA,NaN,Q1'24,ADESA,completed,ADESA-7708
2,"['Shoulder(s)', 'Ankle']",Los Angeles,ADESA,NaN,Q1'24,ADESA,completed,ADESA-7668
3,Knee,ADESA Cincinnati,ADESA,NaN,Q1'24,ADESA,completed,ADESA-7719
4,Finger(s),"Belton, MO",ADESA,NaN,Q1'24,ADESA,completed,ADESA-7530


In [233]:
injury_df_snowflake.describe()

,Injuries,Location,JobFamilyGroup,JobProfile,Quarter,COMPANY,STATUS,CASE_NUMBER
count,2143,2143,2143,1585,2143,2143,2143,2143
unique,307,275,60,204,5,4,6,2143
top,Finger(s),IN Indianapolis IC,RECONDITIONING,Vehicle Transporter,Q1'25,CARVANA,completed,ADESA-7740
freq,248,70,378,212,533,859,723,1


In [ ]:
# # Uncomment when you want to run the script again
# # Write into Snowflake
# db_snowflake_write.insert('injuries',injury_df_snowflake,truncate=True)

In [ ]:
# # Drop the existing table

# db_snowflake_write.drop_table('Injuries')

In [ ]:
# # Table Creation

# db_snowflake_write.create_table('Injuries',
#                   {
#                      'Injuries': ['VARCHAR(100)'], 
#                      'Location': ['VARCHAR(100)'], 
#                      'JobFamilyGroup': ['VARCHAR(100)'], 
#                      'JobProfile': ['VARCHAR(100)'],
#                      'Quarter': ['VARCHAR(1000)'],
#                      'Company': ['VARCHAR(100)'], 
#                      'STATUS': ['VARCHAR(100)'],
#                      'CASE_NUMBER': ['VARCHAR(100)']
#                   })

### Injury df

In [234]:
# Filtering Only for necessary_JobFamilyGroups
injury_df= injury_df_snowflake[injury_df_snowflake['JobFamilyGroup'].isin(necessary_JobFamilyGroups)]

In [235]:
# Counting the number of injuries at the Location-Org Combination
injury_df = injury_df.groupby(['Location','JobFamilyGroup','Quarter']).agg('count').reset_index()

# Final Injury df
injury_df = injury_df[['Location', 'JobFamilyGroup','Quarter','Injuries']]

## Main Dataframe

In [236]:
# Merging for Specific CostCentre
main_df= pd.merge(location_df,injury_df, how='outer', on=['Location','JobFamilyGroup','Quarter'])

# Performance PIPs/CARs

In [238]:
# Performance for Last Quater
performance_sql_df = pd.read_sql_query(f"""

WITH cte AS (
Select * FROM PeopleOps.workday.FactEmployeeReviews
WHERE DateInitiated >= DATEADD(QUARTER, DATEDIFF(QUARTER, 0, '{date_today}') - 1, 0)
AND DateInitiated < DATEADD(QUARTER, DATEDIFF(QUARTER, 0, '{date_today}'), 0) 
AND Status in ('In Process','Successfully Completed','Rescinded') 
AND ReviewCategory in ('Corrective Action','Performance Improvement Plan'))

SELECT c.EmployeeId,c.ReviewCategory,c.ReviewReason,c.ResultingActionRating,c.DateInitiated,eh.StartDateTime,
eh.EndDateTime,eh.JobFamilyGroup,eh.Location,
CONCAT('Q', DATEPART(quarter,c.DateInitiated ), '''', RIGHT(DATEPART(year, c.DateInitiated), 2)) as Quarter   
FROM cte c left join PeopleOps.workday.tblEmployeeHistory eh ON c.EmployeeId=eh.EmployeeId
WHERE c.DateInitiated between eh.StartDateTime and eh.EndDateTime
ORDER BY c.DateInitiated

""",odbcconn)

C:\Users\E102068\AppData\Local\Temp\ipykernel_16284\2269378045.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  performance_sql_df = pd.read_sql_query(f"""


### Snowflake - Performance 

In [239]:
performance_snowflake_df = performance_sql_df.copy()

In [ ]:
# # Uncomment when you need to run it again
# Write into Snowflake
# db_snowflake_write.insert('Performance',performance_snowflake_df)

In [ ]:
# # Drop the existing table

# db_snowflake_write.drop_table('Performance')

In [ ]:
# # Performance Table Creation

# db_snowflake_write.create_table('Performance',
#                   {
#                      'EmployeeId': ['INT','NOT NULL'],
#                      'ReviewCategory': ['VARCHAR(1000)'], 
#                      'ReviewReason': ['VARCHAR(1000)'], 
#                      'ResultingActionRating': ['VARCHAR(1000)'],
#                      'DateInitiated': ['TIMESTAMP_NTZ(9)'], 
#                      'StartDateTime': ['TIMESTAMP_NTZ(9)'],
#                      'EndDateTime' : ['TIMESTAMP_NTZ(9)'],
#                      'JobFamilyGroup': ['VARCHAR(1000)'],
#                      'Location': ['VARCHAR(1000)'],
#                      'Quarter': ['VARCHAR(1000)']
#                   })

### Performance Df

In [240]:
# Filtering for siterisk costcentres
performance_df=performance_sql_df[performance_sql_df['JobFamilyGroup']\
                                      .isin(necessary_JobFamilyGroups)]

In [241]:
# Selecting neccesary columns
performance_df= performance_df.iloc[:,[0,1,-3,-2,-1]]

# Count the number of PIPs/Cars and Unique Employees
performance_df= performance_df.groupby(['Location','JobFamilyGroup','Quarter','ReviewCategory']).\
                                agg(Total_Count=('EmployeeId','count'),EmployeesPIPs_CARs=('EmployeeId','nunique')).reset_index()

In [242]:
# Pivot to divide the PIPs/CARs
performance_df= performance_df.pivot(index=['Location','JobFamilyGroup','Quarter','EmployeesPIPs_CARs'],\
                                     columns=['ReviewCategory'],values='Total_Count').reset_index()



In [243]:
# Total sum of all
performance_df= performance_df.groupby(['Location','JobFamilyGroup','Quarter']).agg('sum').reset_index()

## Overall PIPs/CARs

In [244]:
# Filling all NA with 0 for addition
performance_df=performance_df.fillna(0)

# Sum of ALL PIPs/CARs
performance_df['PIPs & CARs']=performance_df['Performance Improvement Plan']\
                                    +performance_df['Corrective Action']

# Rename
performance_df=performance_df.rename(columns={'Performance Improvement Plan':'PIPs',\
                                              'Corrective Action':'CARs'})

## Merge-Main Df

In [245]:
main_df=pd.merge(main_df,performance_df,on=['Location','Quarter','JobFamilyGroup'],how='outer')

# Turnover/Attrition

In [246]:
# SNOWFLAKE USING WORKER LINEAGE


# SELECT DISTINCT EmployeeId FROM PEOPLEOPS_SANDBOX.DEV.WORKDAY_EMPLOYEE_LINEAGE WHERE DATE='2024-01-01' AND DATA_TYPE='ACTIVE' AND EMPLOYEETYPE='Regular' AND WORKERSTATUS IN ('Active','On Leave');


# Quarter_start_df = db_snowflake.execute("""

# SELECT wl.EMPLOYEEID, wl.date, wl.DATA_TYPE as Status, wl.employeename, wl.jobprofile, wl.costcenter as Organization, sd.location_name as Location,
#     CONCAT('Q', DATE_PART('quarter', wl.Date), '''', RIGHT(DATE_PART('year', wl.Date), 2)) as Quarter
# FROM PEOPLEOPS_SANDBOX.DEV.WORKDAY_EMPLOYEE_LINEAGE wl
# LEFT JOIN SHARED.DW.DIM_EMPLOYEE sd 
#     on wl.employeeid = sd.employee_id_cv
# WHERE wl.Date = DATEADD('quarter', -1, DATE_TRUNC('quarter', CURRENT_TIMESTAMP))
#     AND wl.MATRIXORGANIZATION is not Null
#     AND wl.data_type='ACTIVE'
# ORDER BY wl.Date, wl.EMPLOYEEID;

# """)

In [247]:
# OLD TURNOVER LOGIC

# Turnover_sql_df= db_conn.execute("""

# with active as 
# (
# SELECT 
#     MIN(StartDateTime) as StartDateTime,
#     MAX(EndDateTime) as EndDateTime,
# 	FirstName, 
# 	LastName,
#     EmployeeId
# FROM 
#     PeopleOps.workday.tblEmployeeHistory
# WHERE 
#     (StartDateTime <= DATEADD(QUARTER, DATEDIFF(QUARTER, 0, '{date_today}'), 0) - 1 AND EndDateTime >= DATEADD(QUARTER, DATEDIFF(QUARTER, 0, '{date_today}') - 1, 0))
#     AND ActiveStatus = 1
#     AND SSIS_IsDeleted = 0
#     AND EmployeeType = 'Regular'
# GROUP BY
# 	FirstName, 
# 	LastName,
#     EmployeeId
# )

# , terms as (
# SELECT 
#      distinct EmployeeId,TerminationDate,
# 	 RANK() OVER(PARTITION BY EmployeeId ORDER BY TerminationDate) AS Ranking 
# FROM 
#     PeopleOps.workday.tblEmployeeHistory
# WHERE 
# 	TerminationDate is Not Null AND
#     (TerminationDate <= DATEADD(QUARTER, DATEDIFF(QUARTER, 0, '{date_today}'), 0) - 1 AND TerminationDate >= DATEADD(QUARTER, DATEDIFF(QUARTER, 0, '{date_today}') - 1, 0))

# )

# select a.EmployeeId,StartDateTime,EndDateTime,FirstName,LastName,TerminationDate,Ranking
# from active a left join terms t on a.EmployeeId=t.EmployeeId
# order by a.EmployeeId


# """)
# # If multiple term then we stop at enddatetime
# Turnover_sql_df['EndDateTime']= np.where(Turnover_sql_df['TerminationDate'].isna(),Turnover_sql_df['EndDateTime'],Turnover_sql_df['TerminationDate'])

### Active/Terms

In [249]:
turnover_sql_active= pd.read_sql_query(f"""

SELECT 
    StartDateTime,
    EndDateTime,
    EmployeeId,
    JobFamilyGroup,
    Location
FROM 
    PeopleOps.workday.tblEmployeeHistory
WHERE 
    (StartDateTime <= DATEADD(QUARTER, DATEDIFF(QUARTER, 0, '{date_today}'), 0) - 1 AND EndDateTime >= DATEADD(QUARTER, DATEDIFF(QUARTER, 0, '{date_today}') - 1, 0))
    AND ActiveStatus = 1
    AND SSIS_IsDeleted = 0
    AND EmployeeType = 'Regular'
Order By 
	EmployeeId

""",odbcconn)

C:\Users\E102068\AppData\Local\Temp\ipykernel_16284\980506473.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  turnover_sql_active= pd.read_sql_query(f"""


In [250]:
# Filter for neccessary jobfamily groups
turnover_sql_active=turnover_sql_active[turnover_sql_active['JobFamilyGroup'].isin(necessary_JobFamilyGroups)]

In [251]:
# Extract only date
turnover_sql_active['StartDateTime']=turnover_sql_active['StartDateTime'].dt.date
turnover_sql_active['EndDateTime']=turnover_sql_active['EndDateTime'].dt.date

In [252]:
# Df to store all the dates active for each employee 
turnover_expanded_data=[]

# Iterate over each row
for index, row in turnover_sql_active.iterrows():
    # Generate dates for all the dates between startdate and enddate
    dates_range= pd.date_range(start=row['StartDateTime'],end=row['EndDateTime'])

    for date in dates_range:
        turnover_expanded_data.append({
            'Date': date,
            'EmployeeId':row['EmployeeId'],
            'JobFamilyGroup':row['JobFamilyGroup'],
            'Location':row['Location']
       })

# Convert to data frame
turnover_expanded_df= pd.DataFrame(turnover_expanded_data)

In [253]:
# Building a date list to capture dynamically all the dates in last quarter
from dateutil.relativedelta import relativedelta

# Define the start and end of the previous quarter
if date_today.month < 4:
    start = datetime.date(date_today.year - 1, 10, 1)
    end = datetime.date(date_today.year - 1, 12, 31)
elif date_today.month < 7:
    start = datetime.date(date_today.year, 1, 1)
    end = datetime.date(date_today.year, 3, 31)
elif date_today.month < 10:
    start = datetime.date(date_today.year, 4, 1)
    end = datetime.date(date_today.year, 6, 30)
else:
    start = datetime.date(date_today.year, 7, 1)
    end = datetime.date(date_today.year, 9, 30)


# Create list of dates within the previous quarter
last_quarter_date_list = [(start + datetime.timedelta(days=x)).isoformat() for x in range((end-start).days + 1)]

In [254]:
# to remove dates where we have overlaping dates for each startdatetime and enddatetime
turnover_active_df = turnover_expanded_df.drop_duplicates()

# Filter for dates only in last quater
turnover_active_df = turnover_active_df[turnover_active_df['Date'].isin(last_quarter_date_list)]

C:\Users\E102068\AppData\Local\Temp\ipykernel_16284\903594529.py:5: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  turnover_active_df = turnover_active_df[turnover_active_df['Date'].isin(last_quarter_date_list)]


In [255]:
# Terminations

turnover_sql_terms= pd.read_sql_query(f"""

SELECT 
    distinct EmployeeId,
    CONCAT(FirstName,' ',LastName) as Name,
    JobFamilyGroup,
    Location,
    TerminationDate
FROM 
    PeopleOps.workday.tblEmployeeHistory
WHERE 
	TerminationDate is Not Null 
    AND SSIS_IsDeleted = 0
    AND EmployeeType = 'Regular'
    AND (TerminationDate <= DATEADD(QUARTER, DATEDIFF(QUARTER, 0,'{date_today}'), 0) - 1 
    AND TerminationDate >= DATEADD(QUARTER, DATEDIFF(QUARTER, 0, '{date_today}') - 1, 0))
ORDER BY 
    TerminationDate

""",odbcconn)

C:\Users\E102068\AppData\Local\Temp\ipykernel_16284\4279349736.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  turnover_sql_terms= pd.read_sql_query(f"""


In [256]:
# Converting to Date
turnover_sql_terms['TerminationDate']=pd.to_datetime(turnover_sql_terms['TerminationDate'])

# Neccesary JobFamilyGroups
turnover_sql_terms=turnover_sql_terms[turnover_sql_terms['JobFamilyGroup'].isin(necessary_JobFamilyGroups)]

# Adding Quarter based on Date column
turnover_sql_terms['Quarter'] = turnover_sql_terms['TerminationDate'].dt.to_period("Q")
turnover_sql_terms['Quarter'] = turnover_sql_terms['Quarter'].apply(lambda q: f"{q.strftime('Q%q')}'{str(q.year)[2:]}")

### Snowflake - Terminated Emp

In [ ]:
# #Create Table
# db_snowflake_write.create_table('TermEmployee',
#                   {
#                      'EmployeeId': ['INT','NOT NULL'],
#                      'Name': ['VARCHAR(1000)'], 
#                      'JobFamilyGroup': ['VARCHAR(1000)'], 
#                      'Location': ['VARCHAR(1000)'],
#                      'TerminationDate': ['TIMESTAMP_NTZ(9)'], 
#                      'Quarter': ['VARCHAR(1000)']
#                   })

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://carvana.okta.com/app/snowflake/exkhr8majsTrZPBMF1t7/sso/saml?SAMLRequest=jZJBc9owEIX%2Fikc9Y9nGEKIBMiaUlgmkNBjactvawijYkquVMfTXV5jQSQ%2FJ9KZZvSd9u2%2F7d8cidw5co1ByQHzXIw6XiUqFzAZkFU9aPeKgAZlCriQfkBNHcjfsIxR5yaLK7OQT%2F1VxNI59SCJrLgak0pIpQIFMQsGRmYQto%2FmMBa7HSq2MSlROXlnedwAi18YSXi0pCou3M6ZklNZ17dZtV%2BmMBp7nUe%2BWWtVZ8uGqP9qe3tD71AvPequw8sUL20jIywjew%2Fp5ESH7HMeL1uLLMiZOdEW9VxKrgusl1weR8NXT7AKAluAhisLbdtB2Uap6m8OeJ6ooK2Mfc%2B2JbnlKc5UJ2%2B90PCDlXqTrGYhxmEWjVTb58bs7g0%2B79dfvED37WbR8EPPTx29dKFewmUQJcdbXQINzoFPEik%2FlOUZjS17QaXlhKwjjwGdhj3VuXL%2FT3RBnbGMUEkzjvLImoA8gwVV7Aw0clCX9y035cb%2FTvQKeMdabxWg%2B8c0NRVT0nCq5LAprAPTwP9vv09eml1V7tNOfjhcqF8nJmShdgHk7HN%2F1m4pIW9tGyngBIo%2FSVHNEG1Keq%2FpeczB2o42uOKHDy6%2F%2F7vTwDw%3D%3D&RelayState=ve

,TERMINATIONDATE,EMPLOYEE_ID,NAME,JOB_FAMILY_GROUP,LOCATION,QUARTER


In [259]:
# # Insert Data

# db_snowflake_write.insert('TermEmployee',turnover_sql_terms)

### Location/JobFamilyGroup Calculation

#### Active Emp Calculation

In [260]:
# Count the number of active employees on each day
active_df= turnover_active_df.groupby(['Date','JobFamilyGroup','Location'])\
    .agg(employees=('EmployeeId','count')).reset_index()

# Adding WeekNumber for based on Date column
active_df['WeekNumber']=active_df['Date'].dt.isocalendar().week # Add this for weekly turnover

# Adding Quarter based on Date column
active_df['Quarter'] = active_df['Date'].dt.to_period("Q")
active_df['Quarter'] = active_df['Quarter'].apply(lambda q: f"{q.strftime('Q%q')}'{str(q.year)[2:]}")

In [261]:
# Quarter calculation
# Calculating the average headcount for the Quater
# (Add Week Number in groupby for weekly turnover)

active_qt_df=active_df.groupby(['Quarter','JobFamilyGroup','Location'])\
    .agg(Avg_emp=('employees','mean')).reset_index()

# Rounding the weekly
active_qt_df['Avg_emp']=round(active_qt_df['Avg_emp'],2)

#### Term Calculation

In [262]:
# Count number of terms each day
term_df= turnover_sql_terms.groupby(['TerminationDate','JobFamilyGroup','Location'])\
    .agg(Terminations=('EmployeeId','count')).reset_index()

# Renaming Termination Date
term_df=term_df.rename(columns={'TerminationDate':'Date'})

# Adding Weeknumber based on Termination Date
term_df['WeekNumber']=term_df['Date'].dt.isocalendar().week # Add this for weekly turnover

# Adding Quarter based on Date column
term_df['Quarter'] = term_df['Date'].dt.to_period("Q")
term_df['Quarter'] = term_df['Quarter'].apply(lambda q: f"{q.strftime('Q%q')}'{str(q.year)[2:]}")

In [263]:
# Quarter calculation
# Calculating the terms for the quarter

term_qt_df=term_df.groupby(['Quarter','JobFamilyGroup','Location'])\
    .agg(Terminations=('Terminations','sum')).reset_index()

#### Turnover

In [264]:
# Combining Active and Terms 
turnover_df=pd.merge(active_df,term_df,on=['Date','WeekNumber','Quarter','Location','JobFamilyGroup'],how='left')

# Fill NA
turnover_df=turnover_df.fillna(0)

#### Snowflake- Turnover

In [265]:
## TURNOVER DASHBOARD

#### Quaterly Turnover

In [266]:
# Combining Active and Terms 
turnover_qt_df=pd.merge(active_qt_df,term_qt_df,on=['Quarter','Location','JobFamilyGroup'],how='left')

# Fill NA
turnover_qt_df=turnover_qt_df.fillna(0)

In [267]:
turnover_qt_df= turnover_qt_df.groupby(['Quarter','JobFamilyGroup','Location']).\
                            agg(Avg_emp=('Avg_emp','mean'),\
                                Terminations=('Terminations','sum')).reset_index()

# Calculating the turnover rate 
turnover_qt_df['Quaterly_Turnover']=round((turnover_qt_df['Terminations']/turnover_qt_df['Avg_emp'])*100,2)

In [268]:
turnover_qt_df.head(5)

,Quarter,JobFamilyGroup,Location,Avg_emp,Terminations,Quaterly_Turnover
0,Q1'25,ADESA,NV Las Vegas - Gowan IC,1.00,0.0,0.00
1,Q1'25,Logistics,AL Bessemer IC,56.04,8.0,14.28
2,Q1'25,Logistics,AL Mobile - Theodore,20.77,7.0,33.70
3,Q1'25,Logistics,AR West Memphis IC,40.19,5.0,12.44
4,Q1'25,Logistics,AZ Phoenix - Beck IC,2.33,0.0,0.00


## Merge - Main DF

In [269]:
main_df=pd.merge(main_df,turnover_qt_df,on=['Quarter','JobFamilyGroup','Location'],how='left')

# Removing avg_emp col
main_df.drop('Avg_emp', axis=1, inplace=True)

In [270]:
main_df.head(5)

,Location,Score,eSat Change,JobFamilyGroup,Quarter,Injuries,EmployeesPIPs_CARs,CARs,PIPs,PIPs & CARs,Terminations,Quaterly_Turnover
0,0,0.0,0.0,Post Production,Q1'25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0.0,0.0,Post Production,Q1'25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11625 nino way mira loma ca 91752,NaN,NaN,ADESA,Q3'24,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,ADESA,NaN,NaN,ADESA,Q2'24,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ADESA Cincinnati,NaN,NaN,ADESA,Q1'24,1.0,NaN,NaN,NaN,NaN,NaN,NaN


# Navex - Siterisk 2.0

In [ ]:
## SITE RISK 2.0

# Total Employees- Location/JobProfile

In [ ]:
# OLD Total Emp LOGIC
#Pulling All Locations,Costcentres Required and Total EMployees

# totalemp_df= db_snowflake.execute("""

# SELECT LOCATION_NAME,JOB_FAMILY_GROUP, count(*) as TotalEmployees
# from shared.dw.dim_employee
# where JOB_FAMILY_GROUP in ('RECONDITIONING','MARKET_OPERATIONS','REGULATORY_OPERATIONS','LOGISTICS','WHOLESALE_OPERATIONS')
# AND (is_employee_active='True' or  is_on_loa='True')
# group by LOCATION_NAME,JOB_FAMILY_GROUP
# order by JOB_FAMILY_GROUP,location_name,TotalEmployees desc;

# """)

# # Pulling all Locations, Total Emp for ADESA - due to many misconsistencies with data

# ADESAemp_df= db_snowflake.execute("""

# SELECT LOCATION_NAME,'ADESA' as CostCentre,count(*) as TotalEmployees
# from shared.dw.dim_employee
# where JOB_PROFILE like ('%ADESA%')
# and is_employee_active='True' or  is_on_loa='True'
# group by LOCATION_NAME;

# """)


# Cleaning of Names

# totalemp_df=totalemp_df.rename(columns={'LOCATION_NAME':'Location','JOB_FAMILY_GROUP':'CostCentre','TOTALEMPLOYEES':'TotalEmployees'})

# ADESAemp_df=ADESAemp_df.rename(columns={'LOCATION_NAME':'Location','COSTCENTRE':'CostCentre','TOTALEMPLOYEES':'TotalEmployees'})

# # Rename Coscentre names
# totalemp_df['CostCentre']=totalemp_df['CostCentre'].replace({'RECONDITIONING':'Reconditioning',
#                                        'MARKET_OPERATIONS':'Market Operations',
#                                        'REGULATORY_OPERATIONS':'Regulatory Operations',
#                                        'LOGISTICS':'Logistics',
#                                        'WHOLESALE_OPERATIONS':'Wholesale Operations'})

# # Concat ADESA and TotalEmp
# totalemp_df = pd.concat([totalemp_df,ADESAemp_df]).reset_index(drop=True)

In [ ]:
totalemp_Loc_Jbp_df = db_conn.execute("""

SELECT 
    JobFamilyGroup,
    Location,
    count(*)as total_employees
FROM 
    PeopleOps.workday.tblEmployee
WHERE 
    ActiveStatus = 1
    AND SSIS_IsDeleted = 0
    AND EmployeeType = 'Regular'
	And JobProfile Not like '%ADESA%'
GROUP By
	JobFamilyGroup,
    Location
Order By 
	total_employees desc,
	JobFamilyGroup,
    Location

""")

In [ ]:
total_emp_ADESA= db_conn.execute("""

SELECT 
    'ADESA' as JobFamilyGroup,
    Location,
    count(*)as total_employees
FROM 
    PeopleOps.workday.tblEmployee
WHERE 
    ActiveStatus = 1
    AND SSIS_IsDeleted = 0
    AND EmployeeType = 'Regular'
	AND JobProfile like '%ADESA%'
GROUP By
    Location
Order By 
	total_employees desc,
    Location

""")

In [ ]:
totalemp_Loc_Jbp_df=pd.concat([totalemp_Loc_Jbp_df,total_emp_ADESA])

## Merge- Main df

In [ ]:
main_df= pd.merge(main_df,totalemp_Loc_Jbp_df,on=['Location','JobFamilyGroup'],how='left')

In [ ]:
main_df.columns

## Snowflake - Main DF

In [ ]:
# # Create Table
# db_snowflake_write.create_table('metrics',
#                   {
#                      'Location': ['VARCHAR(1000)'],
#                      'Score': ['INT'],
#                      'eSat Change': ['INT'], 
#                      'JobFamilyGroup': ['VARCHAR(1000)'],
#                      'Quarter': ['VARCHAR(1000)'], 
#                      'Injuries': ['INT'],
#                      'EmployeesPIPs_CARs': ['INT'],
#                      'CARs': ['INT'],
#                      'PIPs': ['INT'],
#                      'PIPs & CARs': ['INT'],
#                      'Terminations': ['INT'],
#                      'Quaterly_Turnover': ['INT'],
#                      'total_employees': ['INT'],
#                   })

In [ ]:
# Inserting the main df
db_snowflake_write.insert('metrics',main_df,truncate=True)

In [ ]:
# db_snowflake_write.drop_table('main_df')

# Normalised Df for ranking - Site risk 2.0

In [ ]:
normalised_df=main_df

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd


# Assuming 'df' is your DataFrame.
scaler = MinMaxScaler()


# Select only the numeric columns in the DataFrame
numeric_cols = normalised_df.select_dtypes(include=[np.number]).columns


# Fit on the data
scaler.fit(normalised_df[numeric_cols])


# Transform the data. This will return a numpy array.
scaled_data = scaler.transform(normalised_df[numeric_cols])


# Create a DataFrame from the numpy array, and assign the original column names
scaled_df = pd.DataFrame(scaled_data, columns=numeric_cols)


# Replace the numeric columns in the original DataFrame with the scaled values
normalised_df[numeric_cols] = scaled_df

In [ ]:
normalised_df.columns

In [ ]:
# First, fill NaN values with 0
normalised_df = normalised_df.fillna(0)

In [ ]:
normalised_df['Siterisk_KPI'] =(normalised_df['Score']*30)\
                                +(normalised_df['Quaterly_Turnover']*30)\
                                +(normalised_df['Terminations']*15)\
                                +(normalised_df['Injuries']*10)\
                                +(normalised_df['PIPs & CARs']*15)                                

In [ ]:
# Select Only Neccesary Columns


# Overall

## Overall Files

In [ ]:
overall_files

In [ ]:
overall_df= pd.DataFrame()
# Appending all the csv files into one df
for file in overall_files:
    df_temp=pd.read_csv(file)
    df_temp['File']= file.split('_')[-1].split('.')[0]
    df_temp['Quarter']=file.split('_')[-2]
        # filename=file.split('_')[-1].split('.')[0]
    overall_df=pd.concat([overall_df,df_temp],ignore_index=True)

overall_df=overall_df.rename(columns={'Organization':'JobFamilyGroup'})

In [ ]:
# Overall CostCentre Dataframe
overall_costcentre_df= overall_df[overall_df['Manager Email'].isna() & overall_df['Location'].isna()]

# Selecting only neccesary columns
overall_costcentre_df= overall_costcentre_df.iloc[:,[0,5,6,10]].reset_index(drop=True)

# Rename Scores 
overall_costcentre_df=overall_costcentre_df.rename(columns={overall_costcentre_df.columns[1]:'Score',overall_costcentre_df.columns[2]:'eSat Change'})
overall_costcentre_df['Location']='All'

In [ ]:
# Overall Location Df
overall_locations_df = overall_df[overall_df['JobFamilyGroup'].isna() & overall_df['Manager Email'].isna()]

# Selecting only neccesary columns
overall_locations_df= overall_locations_df.iloc[:,[-2,5,6,-3]].reset_index(drop=True)

# Rename Scores 
overall_locations_df=overall_locations_df.rename(columns={overall_locations_df.columns[1]:'Score',overall_locations_df.columns[2]:'eSat Change'})
overall_locations_df['JobFamilyGroup']='All'

In [ ]:
# Overall Manager Df
overall_managers_df = overall_df[overall_df['JobFamilyGroup'].isna() & overall_df['Location'].isna()]

# Selecting only neccesary columns
overall_managers_df= overall_managers_df.iloc[:,[-1,5,6,-3]].reset_index(drop=True)

# Rename Scores 
overall_managers_df=overall_managers_df.rename(columns={overall_managers_df.columns[1]:'Score',overall_managers_df.columns[2]:'eSat Change'})

## Managers

In [ ]:
overall_managers_sql= db_conn.execute("""

SELECT 
	distinct Manager1_Name as Manager,
	Manager1_Level as ManagerLevel,
    JobFamilyGroup,
    Location,
    count(*) as Employees
FROM 
    PeopleOps.workday.tblEmployee
WHERE 
    ActiveStatus = 1
    AND SSIS_IsDeleted = 0
    AND EmployeeType = 'Regular'
    AND JobProfile NOT like '%ADESA%'
GROUP By
	Manager1_Name,
	Manager1_Level,
	JobFamilyGroup,
    Location
Order By 
	Employees desc,
	JobFamilyGroup,
    Location

""")

overall_adesa_managers_sql= db_conn.execute("""

SELECT 
	distinct Manager1_Name as Manager,
	Manager1_Level as ManagerLevel,
    'ADESA' as JobFamilyGroup,
    Location,
    count(*) as Employees
FROM 
    PeopleOps.workday.tblEmployee
WHERE 
    ActiveStatus = 1
    AND SSIS_IsDeleted = 0
    AND EmployeeType = 'Regular'
	And JobProfile like '%ADESA%'
GROUP By
	Manager1_Name,
	Manager1_Level,
	JobFamilyGroup,
    Location
Order By 
	Employees desc,
	JobFamilyGroup,
    Location

""")

In [ ]:
# Concat managers df
overall_managers_sql=pd.concat([overall_managers_sql,overall_adesa_managers_sql])

In [ ]:
# Get the managers name
overall_managers_df['ManagerName'] = overall_managers_df['Manager Email'].str.split("'").str[0]

# Striping any space for proper join
overall_managers_df['Manager'] = overall_managers_df['ManagerName'].str.strip()
overall_managers_sql['Manager'] = overall_managers_sql['Manager'].str.strip()
# overall_adesa_managers_sql['Manager']=overall_adesa_managers_sql['Manager'].str.strip()

# Striping in between
overall_managers_df['Manager'] = overall_managers_df['ManagerName'].str.replace(" ","")
overall_managers_sql['Manager'] = overall_managers_sql['Manager'].str.replace(" ","")
# overall_adesa_managers_sql['Manager']=overall_adesa_managers_sql['Manager'].str.replace(" ","")

In [ ]:
# Joining GLINT and SQL
managers_df=pd.merge(overall_managers_df,overall_managers_sql,on='Manager',how='left')

# Selecting only neccesary jobfamily
managers_df=managers_df[managers_df['JobFamilyGroup'].isin(necessary_JobFamilyGroups)]

# Selecting only neccesary columns
managers_df=managers_df[['ManagerName','ManagerLevel','Score','eSat Change','Quarter','Location','JobFamilyGroup','Employees']]

In [ ]:
# Convert to numeric
managers_df['Score']=pd.to_numeric(managers_df['Score'],errors='coerce')
managers_df['eSat Change']=pd.to_numeric(managers_df['eSat Change'],errors='coerce')

# Fill NA
managers_df=managers_df.fillna(0)

### Snowflake - Managers

In [ ]:
# # # Create Table
# db_snowflake_write.create_table('managers',
#                   {
#                      'ManagerName': ['VARCHAR(1000)'],
#                      'ManagerLevel': ['VARCHAR(1000)'],
#                      'Score': ['INT'],
#                      'eSat Change': ['INT'],
#                      'Quarter': ['VARCHAR(1000)'], 
#                      'Location': ['VARCHAR(1000)'], 
#                      'JobFamilyGroup': ['VARCHAR(1000)'],
#                      'Employees': ['VARCHAR(1000)']
#                   })

In [ ]:
# db_snowflake_write.insert('managers',managers_df,truncate=True)

## Overal Emp- CostCentres -(N/A)

In [ ]:
# # Pulling All Locations,Costcentres Required and Total EMployees

# overall_emp_costcentre_df= db_snowflake.execute("""

# SELECT JOB_FAMILY_GROUP as CostCentre, count(*) as TotalEmployees
# from shared.dw.dim_employee
# where JOB_FAMILY_GROUP in ('RECONDITIONING','MARKET_OPERATIONS','REGULATORY_OPERATIONS','LOGISTICS','WHOLESALE_OPERATIONS')
# AND (is_employee_active='True' or  is_on_loa='True')
# group by JOB_FAMILY_GROUP
# order by TotalEmployees desc;

# """)

# # Pulling all Locations, Total Emp for ADESA - due to many misconsistencies with data

# overall_ADESAemp_df= db_snowflake.execute("""

# SELECT 'ADESA' as CostCentre,count(*) as TotalEmployees
# from shared.dw.dim_employee
# where JOB_PROFILE like ('%ADESA%')
# and is_employee_active='True' or  is_on_loa='True';

# """)

In [ ]:
# # Combining the overall employee df
# overall_emp_costcentre_df= pd.concat([overall_emp_costcentre_df,overall_ADESAemp_df])

# # Clearing names
# overall_emp_costcentre_df=overall_emp_costcentre_df.rename(columns={'COSTCENTRE':'CostCentre','TOTALEMPLOYEES':'TotalEmployees'})

# # Rename Coscentre names
# overall_emp_costcentre_df['CostCentre']=overall_emp_costcentre_df['CostCentre'].replace({'RECONDITIONING':'Reconditioning',
#                                        'MARKET_OPERATIONS':'Market Operations',
#                                        'REGULATORY_OPERATIONS':'Regulatory Operations',
#                                        'LOGISTICS':'Logistics',
#                                        'WHOLESALE_OPERATIONS':'Wholesale Operations'})

# # Filtring only for SiteRisk Costcentres
# overall_emp_costcentre_df=overall_emp_costcentre_df[overall_emp_costcentre_df['CostCentre'].isin(SiteRisk_CostCentres)]

## Overall Emp - Locations - (N/A)

In [ ]:
# overall_emp_locations_df= db_snowflake.execute("""

# SELECT LOCATION_NAME,count(*) as TotalEmployees
# from shared.dw.dim_employee
# where (JOB_FAMILY_GROUP in ('RECONDITIONING','MARKET_OPERATIONS','REGULATORY_OPERATIONS','LOGISTICS','WHOLESALE_OPERATIONS')
# AND (is_employee_active='True' or  is_on_loa='True'))
# OR (JOB_PROFILE like ('%ADESA%')) --ADESA Locations
# group by LOCATION_NAME
# order by TotalEmployees desc;

# """)

In [ ]:
# # Clearing names
# overall_emp_locations_df=overall_emp_locations_df.rename(columns={'LOCATION_NAME':'Location','TOTALEMPLOYEES':'TotalEmployees'})

### Overall Locations/CostCentre

In [ ]:
# # Merging Total employees into overall locations/costcentre

# overall_locations_df= pd.merge(overall_locations_df,overall_emp_locations_df, on='Location',how='left')

# overall_costcentre_df= pd.merge(overall_costcentre_df,overall_emp_costcentre_df, on='CostCentre', how='left')

# # Filtering out only for specific costcentre

# overall_costcentre_df=overall_costcentre_df[overall_costcentre_df['CostCentre'].isin(SiteRisk_CostCentres)]
# print(main_df[main_df['CostCentre']=='ADESA']['TotalEmployees'].sum())

# # Overall DF

# overall_main_df = pd.concat([overall_locations_df,overall_costcentre_df])

# # # Convert 'Score' column to numeric, errors='coerce' will replace non-numeric values with NaN
# # overall_main_df['Score'] = pd.to_numeric(overall_main_df['Score'], errors='coerce')
# # overall_main_df['eSat Change'] = pd.to_numeric(overall_main_df['eSat Change'], errors='coerce')

# Automation Bot and Webscraping (N/A)

In [ ]:
# import selenium
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# # Make sure website is open after run
# options = Options()
# options.add_experimental_option("detach",True)

# # Add driver
# driver= webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
# #driver.get("https://app.us1.glintinc.com/carvana/dashboard")
# driver.get("https://www.techwithtim.net/")

# Old Logic / Reference

In [ ]:
# AdHoc
# last_quater_performance_sql_df= db_conn.execute("""

# Select * FROM PeopleOps.workday.FactEmployeeReviews
# WHERE ReviewStartDate >= DATEADD(QUARTER, DATEDIFF(QUARTER, 0, '{date_today}') - 1, 0)
# AND ReviewStartDate < DATEADD(QUARTER, DATEDIFF(QUARTER, 0, '{date_today}'), 0) 
# AND Status in ('In Process','Successfully Completed','Rescinded') 
# AND ReviewCategory in ('Corrective Action','Performance Improvement Plan')

# """)

# not_in_df = last_quater_performance_sql_df[~last_quater_performance_sql_df['EmployeeId'].isin(performance_sql_df['EmployeeId'])]

In [ ]:
# OLD Pips/CARs Logic

# # PIPs and CARs Segmentation
# pips_performance  = performance_sql_df[performance_sql_df['ReviewCategory']=='Performance Improvement Plan']
# pips_performance= pips_performance.iloc[:,[0,1,-2,-1]]


# cars_performance = performance_sql_df[performance_sql_df['ReviewCategory']=='Corrective Action']
# cars_performance = cars_performance.iloc[:,[0,1,-2,-1]]
# pips_performancecheck=pips_performance.groupby(['ReviewCategory','Location','Quarter']).agg('count').reset_index()

In [ ]:
# LOCAL CSV PULL METHOD
# # Files to pull location demographic csv
# subdirectory_path=os.path.join("Glint Reports","Q1'24","Overall")

# # Level 1 csv files
# overall_files= glob.glob(os.path.join(subdirectory_path, "*.csv"))


# # Managers dataframe and selecting only imp columns
# Managers_df= pd.read_csv("Glint Reports\Q1'24\Overall\glint_report_Q1'24_Managers.csv")
# Managers_df['Quarter']= 
# Managers_df=Managers_df.iloc[:,[0,5,6]]

# # CostCentres dataframe and selecting only imp columns
# overall_costcentre_df= pd.read_csv("Glint Reports\Q1'24\Overall\glint_report_Q1'24_CostCentres.csv")
# overall_costcentre_df= overall_costcentre_df.iloc[:,[0,5,6]]

In [ ]:
#OLD Injury Logic

# # Fetching Total Employees to calculate ratio excluding ADESA employee count becaus of ADESA coscentre and the original costcentre
# Total_employees= db_conn.execute("""

# SELECT Count(*) as TotalEmployees ,JobFamilyGroup as CostCentre, Location 
# FROM [PeopleOps].[workday].[tblEmployee] 
# WHERE WorkerStatus In ('Active','On Leave') 
# AND Company NOT LIKE '%ADESA%' 
# GROUP BY JobFamilyGroup,Location 
# ORDER BY JobFamilyGroup,Location

# """)

# Total_employees_Overall= db_conn.execute("""

# SELECT Count(*) as TotalEmployees, Location 
# FROM [PeopleOps].[workday].[tblEmployee] 
# WHERE WorkerStatus In ('Active','On Leave') 
# AND Company NOT LIKE '%ADESA%' 
# AND JobFamilyGroup NOT LIKE '%ADESA%'
# GROUP BY Location 
# ORDER BY TotalEmployees desc,Location

# """)

# # Merging with total employees
# injury_df= pd.merge(injury_df,Total_employees, how='left', on=['Location','CostCentre'])

# # Injuries across all locations
# injury_all_df= injury_df[['Location','Quarter','Injuries']]
# injury_all_df = injury_all_df.groupby(['Location','Quarter']).agg('sum').reset_index()
# injury_all_df= pd.merge(injury_all_df,Total_employees_Overall, how='left', on=['Location'])
# injury_all_df['CostCentre']='LocationsAll'
# # Join Injury Data
# injury_df=pd.concat([injury_df,injury_all_df])

# # Calculating the ratio
# injury_df['Injury Percentage']= round((injury_df['Injuries']/injury_df['TotalEmployees'])*100,2)

In [ ]:
# dfs={}
# for file in files:
#     # Define the file name
#     file_name = pd.Series(file)
#     # file_name = file.split('/')[-1].split('.')[0]

#     # Use str.extract to capture the string from "Q" to just before ".csv"
#     captured_string = file_name.str.extract('(Q.*)(?=\.csv)', expand=False)[0]

#     # Df 
#     dfs[captured_string]=pd.read_csv(file)


# # for i, df_name in enumerate(dfs.keys(), start=1):
# #     globals()[f'df{i}'] = dfs[df_name]

# for key, value in dfs.items():
#     globals()[key] = value

# SANDBOX

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

data = fetch_california_housing()

df = pd.DataFrame(data.data, columns=data.feature_names)
target = pd.DataFrame(data.target, columns=["MedHouseVal"])

In [ ]:
X = df
Y = target["MedHouseVal"]

In [ ]:
X = sm.add_constant(X)

In [ ]:
model = sm.OLS(Y, X).fit()

In [ ]:
print(model.summary())

In [ ]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [ ]:
model_std = sm.OLS(Y, sm.add_constant(X_std)).fit()
print(model_std.summary())

In [ ]:
coef_sq = np.square(model_std.params[1:])
sum_coef_sq = np.sum(coef_sq)
contributions = coef_sq / sum_coef_sq
print(contributions)